In [ ]:
from chengyu.config import settings
from chengyu.dedupe import list_existing_chengyu
from chengyu.gen import gen_unique_episode_strict, script_to_markdown
#from chengyu.cover import draw_cover_png
from chengyu.tts import tts_mp3
from chengyu.publisher import publish_episode
from chengyu.cover_ai import draw_cover_png_story
# ...
# cover_png = draw_cover_png(settings.SHOW_NAME, data["chengyu"], data["pinyin"], data["gloss"])
cover_png = draw_cover_png_story(
    chengyu=data["chengyu"],
    pinyin=data["pinyin"],
    english=data["gloss"],
    script=data["script"],
    show_mark=settings.SHOW_NAME,
    model=settings.IMAGE_MODEL,
    size=settings.IMAGE_SIZE,
    out_size=3000
)


# 0) Already-used idioms (full set)
forbidden = list_existing_chengyu(settings.REPO, settings.GITHUB_BRANCH)

# 1) Strict-unique generation
data = gen_unique_episode_strict(settings.SHOW_NAME, settings.GEN_MODEL, forbidden,
                                 batch_size=20, max_rounds=20)

# 2) Cover image
cover_png = draw_cover_png(settings.SHOW_NAME, data["chengyu"], data["pinyin"], data["gloss"])

# 3) TTS (disable by setting SKIP_TTS=True)
SKIP_TTS = False
audio_mp3 = None if SKIP_TTS else tts_mp3(data["script"], settings.TTS_MODEL, settings.TTS_VOICE)

# 4) Structured Markdown body
body_md = script_to_markdown(data["chengyu"], data["pinyin"], data["gloss"], data["teaser"], data["script"], settings.GEN_MODEL)

# 5) Publish
publish_episode(
    show_name=settings.SHOW_NAME,
    repo=settings.REPO,
    branch=settings.GITHUB_BRANCH,
    site_url=settings.SITE_URL,
    baseurl=settings.BASEURL,
    publish_time_utc=settings.PUBLISH_TIME_UTC,
    data=data,
    body_md=body_md,
    cover_png=cover_png,
    audio_mp3=audio_mp3,
    dry_run=settings.DRY_RUN,
)


+ git clone --depth 1 --branch main *** /var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_seen_b5i6cahx


Cloning into '/var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_seen_b5i6cahx'...


+ git clone --depth 1 --branch main *** /var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_pub_czpmh9q_


Cloning into '/var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_pub_czpmh9q_'...


+ git config user.name Chengyu Publisher Bot
+ git config user.email actions@users.noreply.github.com
+ git clone --depth 1 --branch main *** /var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_seen_rksqu39d


Cloning into '/var/folders/wn/z035vl9x1_q0_8dhkvn9n24h0000gn/T/chengyu_seen_rksqu39d'...


+ git add .
+ git commit -m Add episode 2025-08-25-bei-gong-she-ying
[main 7ae5a55] Add episode 2025-08-25-bei-gong-she-ying
 5 files changed, 48 insertions(+)
 create mode 100644 _posts/2025-08-25-bei-gong-she-ying.md
 create mode 100644 episodes/2025-08-25-bei-gong-she-ying/audio.mp3
 create mode 100644 episodes/2025-08-25-bei-gong-she-ying/cover.png
 create mode 100644 episodes/2025-08-25-bei-gong-she-ying/metadata.json
 create mode 100644 episodes/2025-08-25-bei-gong-she-ying/transcript.txt
+ git push origin main
✔ Pushed one commit. Pages workflow will build & deploy.


To https://github.com/kohlenberg/chengyudaily.git
   f2c4b48..7ae5a55  main -> main


{'folder': '2025-08-25-bei-gong-she-ying',
 'post': '2025-08-25-bei-gong-she-ying.md'}